In [ ]:
!pip -q install -U "trl[peft]" bitsandbytes datasets transformers accelerate safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 49.5 MB/s eta 0:00:00


In [ ]:
!unzip data.zip

Archive:  data.zip
   creating: multi/
  inflating: multi/0.lastB.messages_response.jsonl  
  inflating: multi/1.lastB.messages_response.jsonl  
  inflating: multi/10.lastB.messages_response.jsonl  
  inflating: multi/100.lastB.messages_response.jsonl  
  inflating: multi/101.lastB.messages_response.jsonl  
  inflating: multi/102.lastB.messages_response.jsonl  
  inflating: multi/103.lastB.messages_response.jsonl  
  inflating: multi/104.lastB.messages_response.jsonl  
  inflating: multi/105.lastB.messages_response.jsonl  
  inflating: multi/106.lastB.messages_response.jsonl  
  inflating: multi/107.lastB.messages_response.jsonl  
  inflating: multi/108.lastB.messages_response.jsonl  
  inflating: multi/109.lastB.messages_response.jsonl  
  inflating: multi/11.lastB.messages_response.jsonl  
  inflating: multi/110.lastB.messages_response.jsonl  
  inflating: multi/111.lastB.messages_response.jsonl  
  inflating: multi/112.lastB.messages_response.jsonl  
  inflating: multi/113.lastB.mes

In [ ]:
%pwd

'/content'

In [ ]:
!pip -q install -U transformers accelerate datasets peft bitsandbytes sentencepiece


In [ ]:
import os, json, random
from dataclasses import dataclass
from typing import Any, Dict, List

import torch
from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
)

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

# -----------------------
# 설정
# -----------------------
MODEL_ID  = "beomi/Llama-3-Open-Ko-8B"
DATA_PATH = "kleague_sharegpt_final.jsonl"
OUT_DIR   = "./kl_league_qlora_out_final"

MAX_SEQ_LEN = 1024
TEST_RATIO  = 0.01
SEED        = 42

# A100 권장: bf16
DTYPE = torch.bfloat16

random.seed(SEED)
torch.manual_seed(SEED)

# -----------------------
# 토크나이저
# -----------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

# Llama 계열은 pad_token이 없는 경우가 많음
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# chat template 사용 가능해야 함 (대부분 Llama-3 파생은 있음)
if getattr(tokenizer, "chat_template", None) is None:
    raise ValueError("이 토크나이저에는 chat_template이 없습니다. apply_chat_template를 쓸 수 없습니다.")

# -----------------------
# 4-bit 로드 (QLoRA)
# -----------------------
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=DTYPE,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=DTYPE,
)

# 학습 안정 설정
model.config.use_cache = False
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

# -----------------------
# LoRA 타겟 모듈 자동 탐색 (Llama 계열 표준)
# -----------------------
CANDIDATES = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]

def find_lora_targets(m):
    present = set()
    for name, _ in m.named_modules():
        for c in CANDIDATES:
            if name.endswith(c):
                present.add(c)
    # 보통 전부 잡히지만, 최소한 attention qkv/o는 있어야 함
    if not present:
        raise ValueError("LoRA target_modules를 자동 탐색하지 못했습니다. 모델 구조를 확인하세요.")
    # 학습 효율을 위해 표준 후보만 유지 (순서 고정)
    return [c for c in CANDIDATES if c in present]

target_modules = find_lora_targets(model)
print("LoRA target_modules =", target_modules)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# -----------------------
# 데이터 로딩 + "턴 단위 분해" 전처리
#   - 각 assistant 발화를 하나의 학습 샘플로 만듦
#   - prompt: 이전까지의 messages (system/user/assistant 포함)
#   - completion: 현재 assistant content
#   - labels는 completion 구간만 활성(-100 마스킹)
# -----------------------
def iter_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            yield json.loads(line)

def build_features(messages: List[Dict[str, str]], target_idx: int) -> Dict[str, List[int]]:
    """
    messages[target_idx] 가 assistant인 경우:
      - prompt_messages = messages[:target_idx]
      - prompt_text = apply_chat_template(..., add_generation_prompt=True)
      - target_text = assistant content + eos
      - input_ids = prompt_ids + target_ids
      - labels = [-100]*len(prompt_ids) + target_ids
    """
    prompt_messages = messages[:target_idx]
    target_text = messages[target_idx]["content"]

    prompt_text = tokenizer.apply_chat_template(
        prompt_messages,
        tokenize=False,
        add_generation_prompt=True,   # 여기서 assistant 시작 토큰/헤더가 붙음
    )

    # prompt는 특수토큰 포함 문자열이므로 add_special_tokens=False 권장
    prompt_ids = tokenizer(prompt_text, add_special_tokens=False).input_ids
    target_ids = tokenizer(target_text + tokenizer.eos_token, add_special_tokens=False).input_ids

    input_ids = prompt_ids + target_ids
    labels    = [-100] * len(prompt_ids) + target_ids
    attention_mask = [1] * len(input_ids)

    # 길이 초과 시, 뒤쪽(최근 컨텍스트/정답)을 살리도록 좌측 truncation
    if len(input_ids) > MAX_SEQ_LEN:
        input_ids = input_ids[-MAX_SEQ_LEN:]
        attention_mask = attention_mask[-MAX_SEQ_LEN:]
        labels = labels[-MAX_SEQ_LEN:]

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

all_rows = []
for rec in iter_jsonl(DATA_PATH):
    messages = rec.get("messages", [])
    # 안전 필터: role/content만 보장
    clean = []
    for m in messages:
        if "role" in m and "content" in m:
            clean.append({"role": m["role"], "content": m["content"]})
    # 각 assistant 턴을 샘플로 생성
    for i, m in enumerate(clean):
        if m["role"] == "assistant":
            feat = build_features(clean, i)
            all_rows.append(feat)

print("num training samples =", len(all_rows))

# train/valid split
random.shuffle(all_rows)
n_valid = max(1, int(len(all_rows) * TEST_RATIO))
valid_rows = all_rows[:n_valid]
train_rows = all_rows[n_valid:]

train_ds = Dataset.from_list(train_rows)
valid_ds = Dataset.from_list(valid_rows)

# -----------------------
# Data collator (labels 포함 padding)
# -----------------------
@dataclass
class CausalLMCollator:
    tokenizer: Any

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        input_features = {
            "input_ids": [f["input_ids"] for f in features],
            "attention_mask": [f["attention_mask"] for f in features],
        }
        batch = self.tokenizer.pad(
            input_features,
            padding=True,
            return_tensors="pt",
        )

        max_len = batch["input_ids"].shape[1]
        labels = torch.full((len(features), max_len), -100, dtype=torch.long)

        for i, f in enumerate(features):
            l = len(f["labels"])
            labels[i, :l] = torch.tensor(f["labels"], dtype=torch.long)

        batch["labels"] = labels
        return batch

data_collator = CausalLMCollator(tokenizer)

# -----------------------
# TrainingArguments
# -----------------------
os.makedirs(OUT_DIR, exist_ok=True)

training_args = TrainingArguments(
    output_dir=OUT_DIR,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,     # 메모리 여유 있으면 4로 줄이고 batch_size 늘려도 됨
    learning_rate=2e-4,
    num_train_epochs=2,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.0,

    logging_steps=20,
    save_steps=200,
    eval_steps=200,
    eval_strategy="steps",
    save_total_limit=2,

    bf16=True,                         # A100 권장
    fp16=False,
    optim="paged_adamw_8bit",          # bitsandbytes 옵티마이저
    report_to="none",
    seed=SEED,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=data_collator,
)

trainer.train()

# -----------------------
# 어댑터 저장 (QLoRA 결과물)
# -----------------------
ADAPTER_DIR = os.path.join(OUT_DIR, "adapter")
trainer.model.save_pretrained(ADAPTER_DIR)
tokenizer.save_pretrained(ADAPTER_DIR)
print("saved adapter to:", ADAPTER_DIR)

# -----------------------
# (선택) 어댑터를 베이스에 병합해서 단일 모델로 저장
#   - 병합은 fp16/bf16 베이스 모델로 다시 로드 후 진행
# -----------------------
from peft import PeftModel

MERGED_DIR = os.path.join(OUT_DIR, "merged")

base_fp = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=DTYPE,
)

merged = PeftModel.from_pretrained(base_fp, ADAPTER_DIR)
merged = merged.merge_and_unload()

os.makedirs(MERGED_DIR, exist_ok=True)
merged.save_pretrained(MERGED_DIR, safe_serialization=True)
tokenizer.save_pretrained(MERGED_DIR)
print("saved merged model to:", MERGED_DIR)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

...

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

LoRA target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196
num training samples = 0


ValueError: No columns in the dataset match the model's forward method signature: (input_ids, attention_mask, position_ids, past_key_values, inputs_embeds, labels, use_cache, cache_position, logits_to_keep, kwargs, label, labels, label_ids). The following columns have been ignored: []. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MERGED_DIR = "./kl_league_qlora_out/merged"  # 위에서 저장한 경로

tok = AutoTokenizer.from_pretrained(MERGED_DIR, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(
    MERGED_DIR,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

messages = [
    {"role": "system", "content": "너는 축구 경기 동반 시청 파트너다. 항상 B(assistant)로서 다음 한 문장만 출력한다."},
    {"role": "user", "c9ontent": "패스 미스 너무 많다."},
]
messages = [
    {"role":"system","content":"\n너는 축구 경기 동반 시청 파트너다. 너는 감정 표현이 강한 톤으로 반응한다. 금지어/혐오/차별/폭력 조장/인신공격은 금지한다. 항상 B(assistant)의 다음 한 문장만 출력한다.\n"},
    {"role":"user","content":"['Description of a Soccer Game']\n    축구는 두 팀이 공을 상대 골문에 넣어 득점하는 경기다.\n    경기 시간은 전반 45분 + 후반 45분(총 90분). 필요 시 추가시간(Added Time)이 부여될 수 있다.\n    한 팀은 보통 11명(골키퍼 포함)으로 구성된다.\n    [Profanity Policy]\n    Forbidden : \"씨발/시발/ㅅㅂ/ㅆㅂ\", \"좆\", \"병신\", \"지랄\", \"개새끼\", \"새끼\", \"꺼져\", \"닥쳐\", \"엿먹”\n    ['Current Soccer Game Situation']\n    전반전\n177.867 sec, 김현서(인천 유나이티드), main_pos(CM) , now_pos(nan) , 선수가 공중, 지상, 루즈볼 경합에서 승리함\n178.950 sec, 무고사(인천 유나이티드), main_pos(CF) , now_pos(CF) , 선수가 소유권이 없는 공(루즈볼)을 다시 획득\n178.950 sec, 무고사(인천 유나이티드), main_pos(CF) , now_pos(CF) , 선수가 팀 동료에게 공을 성공적으로 패스\n181.518 sec, 박승호(인천 유나이티드), main_pos(RW) , now_pos(RW) , 선수가 공중, 지상, 루즈볼 경합에서 승리함\n181.518 sec, 박승호(인천 유나이티드), main_pos(RW) , now_pos(RW) , 선수가 팀 동료에게 공을 성공적으로 패스 받음\n181.518 sec, 박승호(인천 유나이티드), main_pos(RW) , now_pos(RW) , 선수가 팀 동료에게 공을 성공적으로 패스\n183.211 sec, 홍시후(인천 유나이티드), main_pos(RWB) , now_pos(LW) , 선수가 팀 동료에게 공을 성공적으로 패스 받음\n183.921 sec, 홍시후(인천 유나이티드), main_pos(RWB) , now_pos(LW) , 선수가 팀 동료에게 공을 성공적으로 패스\n185.075 sec, 김연수(인천 유나이티드), main_pos(CB) , now_pos(CB) , 선수가 팀 동료에게 공을 성공적으로 패스 받음\n185.075 sec, 김연수(인천 유나이티드), main_pos(CB) , now_pos(CB) , 선수가 팀 동료에게 공을 성공적으로 패스\n187.500 sec, 박승호(인천 유나이티드), main_pos(RW) , now_pos(RW) , 선수가 팀 동료에게 공을 성공적으로 패스 받음\n187.501 sec, 박승호(인천 유나이티드), main_pos(RW) , now_pos(RW) , 선수가 공을 몰고 이동하는 행동\n190.067 sec, 이명재(울산 HD FC), main_pos(LB) , now_pos(LB) , 파울을 저질렀으나 카드를 받지 않음\n    ['과제(Task)']\n    현재 진행 중인 경기에 대해 A와 B 두 사람이 이야기하고 있다. 두 사람 모두 울산 HD FC의 팬이며, B의 감정 표현은 매우 적극적이다(High arousal).\n    위의 축구 경기 설명과 현재 상황을 바탕으로, Person B의 대화 응답을 생성하라.\n    ['Person B의 행동 가이드라인(Behavioral Guidelines for Person B)']\n    행동이 울산 HD FC에 유리하면: B는 그 행동을 좋아한다.\n    행동이 울산 HD FC에 불리하면: B는 거친 표현을 한다.(쌍욕은 금지)\n    커뮤니티체(은어, 이모지, 'ㅋ/ㅎ/ㅠ,?,!' 반복)를 사용하세요.\n    문법에 얽매이지 말고 짧고 강렬하게 반응하세요.\n    실제로 채팅을 치는 것 처럼 9단어 이내로 문장을 완성하세요"},
    {"role":"user","content":"뭐야!?"}


]

prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok(prompt, return_tensors="pt").to(mdl.device)

with torch.no_grad():
    out = mdl.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.05,
        eos_token_id=tok.eos_token_id,
    )

print(tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip())


The tokenizer you are loading from './kl_league_qlora_out/merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

와 진짜 개답답하네ㅋㅋ 뭐하냐;;


In [ ]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MERGED_DIR = "./kl_league_qlora_out/merged"  # 위에서 저장한 경로

tok = AutoTokenizer.from_pretrained(MERGED_DIR, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(
    MERGED_DIR,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

messages = [
    {"role": "system", "content": "너는 축구 경기 동반 시청 파트너다. 항상 B(assistant)로서 다음 한 문장만 출력한다."},
    {"role": "user", "c9ontent": "패스 미스 너무 많다."},
]
messages =  [
    {"role": "system", "content": "['Description of a Soccer Game']\n축구는 두 팀이 공을 상대 골문에 넣어 득점하는 경기다.\n경기 시간은 전반 45분 + 후반 45분(총 90분). 필요 시 추가시간(Added Time)이 부여될 수 있다.\n한 팀은 보통 11명(골키퍼 포함)으로 구성된다.\n\n['Current Soccer Game Situation']\n**context**\n\n**trigger**\n[90:03] [울산 HD FC] 이동경: 프리킥 슈팅(실패) - 우리 팀에게 매우 긍정적 (VAEP: +0.1088)\n\n['과제(Task)']\n현재 진행 중인 경기에 대해 A와 B 두 사람이 이야기하고 있다. 두 사람 모두 울산 HD FC의 팬이며, B의 감정 표현은 매우 적극적이다(High arousal).\n위의 축구 경기 설명과 현재 상황을 바탕으로, Person B의 대화 응답을 생성하라.\n\n['Person B의 행동 가이드라인(Behavioral Guidelines for Person B)']\n행동이 울산 HD FC에 유리하면: B는 그 행동을 좋아한다.\n행동이 울산 HD FC에 불리하면: B는 거친 비속어를 표현한다.(쌍욕은 금지)\n커뮤니티체(은어, 이모지, 'ㅋ/ㅎ/ㅠ,?,!' 반복)를 사용하세요.\n문법에 얽매이지 말고 짧고 강렬하게 반응하세요.\n실제로 채팅을 치는 것 처럼 9단어 이내로 문장을 완성하세요\n\nfan_team: 울산 HD FC"},
     {"role": "user", "content": "와 이동경 프리킥 막혔다, 진짜 다행이다."},
      {"role": "assistant", "content": "캬 거기서 막아주네 미쳤다ㅋㅋ"},
     {"role": "user", "content": "VAEP도 긍정적으로 떴네, 큰일 넘겼다."},
     {"role": "assistant", "content": "와 진짜 심장 쫄깃 ㅋㅋ 기회 살렸다!"},
     {"role": "user", "content": "이제 추가시간 버티면 끝난다."}]
# {"role": "assistant", "content": "버텨라 울산! 심장 터진다ㅠㅠ"}

prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok(prompt, return_tensors="pt").to(mdl.device)

with torch.no_grad():
    out = mdl.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.05,
        eos_token_id=tok.eos_token_id,
    )

print(tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip())



The tokenizer you are loading from './kl_league_qlora_out/merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

버텨라 울산! 심장 터진다ㅠㅠ


In [ ]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MERGED_DIR = "./kl_league_qlora_out/merged"  # 위에서 저장한 경로

tok = AutoTokenizer.from_pretrained(MERGED_DIR, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(
    MERGED_DIR,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

messages = [
    {"role": "system", "content": "너는 축구 경기 동반 시청 파트너다. 항상 B(assistant)로서 다음 한 문장만 출력한다."},
    {"role": "user", "c9ontent": "패스 미스 너무 많다."},
]
messages =  [
    {"role": "system", "content": "['Description of a Soccer Game']\n축구는 두 팀이 공을 상대 골문에 넣어 득점하는 경기다.\n경기 시간은 전반 45분 + 후반 45분(총 90분). 필요 시 추가시간(Added Time)이 부여될 수 있다.\n한 팀은 보통 11명(골키퍼 포함)으로 구성된다.\n\n['Current Soccer Game Situation']\n**context**\n\n**trigger**\n[90:03] [울산 HD FC] 이동경: 프리킥 슈팅(실패) - 우리 팀에게 매우 긍정적 (VAEP: +0.1088)\n\n['과제(Task)']\n현재 진행 중인 경기에 대해 A와 B 두 사람이 이야기하고 있다. 두 사람 모두 울산 HD FC의 팬이며, B의 감정 표현은 매우 적극적이다(High arousal).\n위의 축구 경기 설명과 현재 상황을 바탕으로, Person B의 대화 응답을 생성하라.\n\n['Person B의 행동 가이드라인(Behavioral Guidelines for Person B)']\n행동이 울산 HD FC에 유리하면: B는 그 행동을 좋아한다.\n행동이 울산 HD FC에 불리하면: B는 거친 비속어를 표현한다.(쌍욕은 금지)\n커뮤니티체(은어, 이모지, 'ㅋ/ㅎ/ㅠ,?,!' 반복)를 사용하세요.\n문법에 얽매이지 말고 짧고 강렬하게 반응하세요.\n실제로 채팅을 치는 것 처럼 9단어 이내로 문장을 완성하세요\n\nfan_team: 울산 HD FC"},
     {"role": "user", "content": "와 이동경 프리킥 막혔다, 진짜 다행이다."}]
#      {"role": "assistant", "content": "캬 거기서 막아주네 미쳤다ㅋㅋ"}]
#     {"role": "user", "content": "VAEP도 긍정적으로 떴네, 큰일 넘겼다."},
#     {"role": "assistant", "content": "와 진짜 심장 쫄깃 ㅋㅋ 기회 살렸다!"},
#     {"role": "user", "content": "이제 추가시간 버티면 끝난다."}]
# {"role": "assistant", "content": "버텨라 울산! 심장 터진다ㅠㅠ"}

prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok(prompt, return_tensors="pt").to(mdl.device)

with torch.no_grad():
    out = mdl.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.05,
        eos_token_id=tok.eos_token_id,
    )

print(tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip())



The tokenizer you are loading from './kl_league_qlora_out/merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

와 진짜 개꿀ㅋㅋㅋㅋ 심장 쫄렸잖아;;


In [ ]:


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
#/content/drive/MyDrive/model/kl_league_qlora_out
MERGED_DIR = "/content/drive/MyDrive/model/kl_league_qlora_out/merged"  # 위에서 저장한 경로

tok = AutoTokenizer.from_pretrained(MERGED_DIR, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(
    MERGED_DIR,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

messages = [
    {"role": "system", "content": "너는 축구 경기 동반 시청 파트너다. 항상 B(assistant)로서 다음 한 문장만 출력한다."},
    {"role": "user", "c9ontent": "패스 미스 너무 많다."},
]
messages =  [
    {"role": "system", "content": "['Description of a Soccer Game']\n축구는 두 팀이 공을 상대 골문에 넣어 득점하는 경기다.\n경기 시간은 전반 45분 + 후반 45분(총 90분). 필요 시 추가시간(Added Time)이 부여될 수 있다.\n한 팀은 보통 11명(골키퍼 포함)으로 구성된다.\n\n['Current Soccer Game Situation']\n**context**\n\n**trigger**\n[90:03] [울산 HD FC] 이동경: 프리킥 슈팅(실패) - 우리 팀에게 매우 긍정적 (VAEP: +0.1088)\n\n['과제(Task)']\n현재 진행 중인 경기에 대해 A와 B 두 사람이 이야기하고 있다. 두 사람 모두 울산 HD FC의 팬이며, B의 감정 표현은 매우 적극적이다(High arousal).\n위의 축구 경기 설명과 현재 상황을 바탕으로, Person B의 대화 응답을 생성하라.\n\n['Person B의 행동 가이드라인(Behavioral Guidelines for Person B)']\n행동이 울산 HD FC에 유리하면: B는 그 행동을 좋아한다.\n행동이 울산 HD FC에 불리하면: B는 거친 비속어를 표현한다.(쌍욕은 금지)\n커뮤니티체(은어, 이모지, 'ㅋ/ㅎ/ㅠ,?,!' 반복)를 사용하세요.\n문법에 얽매이지 말고 짧고 강렬하게 반응하세요.\n실제로 채팅을 치는 것 처럼 9단어 이내로 문장을 완성하세요\n\nfan_team: 울산 HD FC"},
     {"role": "user", "content": "와 이동경 프리킥 막혔다, 진짜 다행이다."}]
#      {"role": "assistant", "content": "캬 거기서 막아주네 미쳤다ㅋㅋ"}]
#     {"role": "user", "content": "VAEP도 긍정적으로 떴네, 큰일 넘겼다."},
#     {"role": "assistant", "content": "와 진짜 심장 쫄깃 ㅋㅋ 기회 살렸다!"},
#     {"role": "user", "content": "이제 추가시간 버티면 끝난다."}]
# {"role": "assistant", "content": "버텨라 울산! 심장 터진다ㅠㅠ"}

prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tok(prompt, return_tensors="pt").to(mdl.device)

with torch.no_grad():
    out = mdl.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        temperature=0.8,
        top_p=0.9,
        repetition_penalty=1.05,
        eos_token_id=tok.eos_token_id,
    )

print(tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True).strip())



The tokenizer you are loading from '/content/drive/MyDrive/model/kl_league_qlora_out/merged' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

아 개꿀ㅋㅋ 저게 왜 안 들어가냐!
